### Versions of libraries used
numpy==1.24.3
pandas==1.3.4
rapidfuzz==3.6.1
ipywidgets==8.1.1

### Output File path: 
/dbfs/FileStore/tables/NORMALIZED_ATTRIBUTE_OUTPUTS

In [0]:
%pip install -U pip
%pip install -U ipywidgets==8.1.1
%pip install -U numpy==1.24.3
%pip install -U rapidfuzz==3.6.1
%pip install -U pandas==1.3.4

import pandas as pd, numpy as np
import os
import io
import json
import datetime as dt
from rapidfuzz import process, fuzz,utils
from ipywidgets import widgets
import datetime as dt

np.set_printoptions(threshold=np.inf)
pd.set_option('chained_assignment',None)
### List of departments applicable for each attribute type
attr_dpt_map = {'Color':['Kitchen & Dining', 'Home Décor' ,'Floral'],\
                'Flavor': ['Pantry','Beverages','Frozen Foods','Candy'],\
                'Scent':['Cleaning and Household','Beauty','Personal Care']}

attb_dpt_combos = []
for attb in attr_dpt_map.keys():
    for i in attr_dpt_map[attb]:
        attb_dpt_combos.append((attb,i))

### Substitutions to be made within the input text before performing the fuzzy match
replacements = {          
 ',': ' ',
 '/': ' ',
 '>':' ',
'<': ' ',
'COFFEE TABLE':'TABLE'}

def remove_substrings(flav_list):
    flav_df = pd.DataFrame(flav_list, columns=['Sentence', 'Scores'])
    flav_df['Word Length'] = flav_df.Sentence.apply(lambda x: len(x.split(' ')))
    flav_df.sort_values('Word Length', inplace= True)
    flav_df.reset_index(drop = True, inplace= True)
    filtered_flav_df = flav_df[flav_df.Scores==100]
    if len(filtered_flav_df.index)==0:
        return flav_df.Sentence.tolist()
    if len(filtered_flav_df.index)==1:
        return filtered_flav_df.Sentence.tolist()
    filtered_flav_df['Drop'] = ''
    for x in range(len(filtered_flav_df)):
        str_1 = filtered_flav_df.Sentence.iloc[x]
        if x==len(filtered_flav_df):
            break
        for sent in filtered_flav_df.Sentence.iloc[x+1:]:
            if str_1 in sent:
                filtered_flav_df.Drop.iloc[x] = 'Yes'
                break
            if len(flav_df) < 2:
                break
    return filtered_flav_df[filtered_flav_df.Drop != 'Yes'].Sentence.tolist()

# Match using 'token set ratio' with threshold
def attribute_fetch(item, color_list_org, subs,cutoff = 85):
    color_list = color_list_org 
    color_no_spaces = [ i for i in color_list_org if not '' in i]
    color_with_spaces = [ i for i in color_list_org if i not in color_no_spaces]
    abbrev_maps = {}
    for i in color_no_spaces:
        for j in color_with_spaces:
            if j.replace(' ','') == i:
                abbrev_maps[i] = j
    matched = process.extract(item, color_list, score_cutoff= cutoff, scorer = fuzz.token_set_ratio, processor=utils.default_process, limit= 7)
    color_shortlist = [i[0] for i in matched]
    matched = [i for i in matched if i[0] in color_shortlist]
    if len(color_shortlist) > 0:
        max_score = matched[0][1]
        final_match = [i for i in matched if i[1]== max_score]
        match_list = [i[0] for i in final_match]
        match_list = [abbrev_maps[i] if i in abbrev_maps.keys() else i for i in match_list]
        match_list = remove_substrings(zip(match_list, [100]*len(match_list)))
        match_list = [subs[i] if i in subs.keys() else i for i in match_list]        
        rematched = process.extract(item, match_list,  scorer = fuzz.partial_ratio, processor=utils.default_process, limit = 5)
        re_max_score = rematched[0][1]
        final_rematch = [ i for i in rematched if i[1]== re_max_score]
        match_list =  remove_substrings([i[0:2] for i in final_rematch])
        match_list_ordered =[]
        for word in item.split(' '): # Ordering scents according to their appearance in the description text
            for color_token in match_list:
                if word in color_token:
                    match_list_ordered.append(color_token)
                    continue
        if len(match_list_ordered) > 0:
            match_list = pd.Series(match_list_ordered).drop_duplicates().tolist()
        else:
            match_list = pd.Series(match_list).drop_duplicates().tolist()
        matched = ';'.join(list(pd.Series(match_list)))
    else: 
        matched = 'OTHER'
    return matched

'''
### Select an Attribute
print('Choose the latest input file to run')
inputs_dir = '/dbfs/FileStore/tables/NORMALIZED_ATTRIBUTE_INPUTS/'
InputList =  os.listdir(inputs_dir)
input_w = widgets.Dropdown(
    options=InputList,
    value=InputList[0],
    description='Attribute:',
    disabled=False,
)
display(input_w)'''

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.3.1
    Uninstalling pip-22.3.1:
      Successfully uninstalled pip-22.3.1
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/139.4 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━ 133.1/139.4 kB 6.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/214.9 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━ 204.8/214.9 kB 11.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━ 204.8/214.9 kB 11.8 MB/s eta 0:00:01
  

"\n### Select an Attribute\nprint('Choose the latest input file to run')\ninputs_dir = '/dbfs/FileStore/tables/NORMALIZED_ATTRIBUTE_INPUTS/'\nInputList =  os.listdir(inputs_dir)\ninput_w = widgets.Dropdown(\n    options=InputList,\n    value=InputList[0],\n    description='Attribute:',\n    disabled=False,\n)\ndisplay(input_w)"

In [0]:
def newest(path): ##Find the newest file in the directory
    files = os.listdir(path)
    paths = [os.path.join(path, basename) for basename in files]
    return max(paths, key=os.path.getctime)

input_path = '/dbfs/FileStore/tables/NORMALIZED_ATTRIBUTE_INPUTS/'
newest_file = newest(input_path)
print('Loading the latest input file:',newest_file)
input_df = pd.read_csv(newest_file)

attb_dpt_inputs = {}
for i in attb_dpt_combos:
    attb = i[0]
    dpt = i[1]
    input_dpt = input_df[input_df.DEPARTMENT == dpt]
    if dpt == 'Home Décor':
        rugs = input_dpt[input_dpt.COMMODITY == 'Rugs']
        dpt = 'Home_Decor_Rugs'
        attb_dpt_inputs[(attb, dpt)] = rugs
        no_rugs = input_dpt[input_dpt.COMMODITY != 'Rugs']
        dpt = 'Home_Decor_NoRugs'
        attb_dpt_inputs[(attb, dpt)] =  no_rugs
    elif dpt == 'Frozen Foods':
        dpt = 'Ice Cream'
        attb_dpt_inputs[(attb, dpt)] =  input_dpt
    else: attb_dpt_inputs[(attb, dpt)] = input_dpt

outputs = []
for keys in attb_dpt_inputs.keys():
    attrib = keys[0]
    select_dpt = keys[1]
    items = attb_dpt_inputs[keys]
    configs_dir = '/dbfs/FileStore/tables/DATA_SCIENCE/FuzzyConfigs/'
    configsList =  os.listdir(configs_dir)
    configfile = [i for i in configsList if ((select_dpt.replace(' ','') in i) | (select_dpt.replace(' ', '').replace('&','and') in i) ) and attrib.upper() in i][0]

    ### Load configs
    with open(configs_dir + configfile, "r") as read_file:
        configs = json.load(read_file)
    print('Completed loading up configs.')

    ### Extract the configs
    attb_list = configs['Normalized_list']
    attb_subs = configs['Substitutions']

    ########################################################################################################################

    def remove_scentlist(itm):
        for iter in itm:
            if iter in attb_list: attb_list.remove(iter)

    if select_dpt == 'Personal Care': 
        print(select_dpt)
        remove_scentlist(['RELAX','CALMING','CALM','OATMEAL','REGULAR','HERBAL','BERRY','CLASSIC','SEA SALT'])
    remove_scentlist(['SUGAR','SWEET','SALT', 'CHOCO', 'WATERMELON'])
    attb_list += ['PNUT BUTTER', 'BBQ', 'PBUTTER', 'PMPKIN', 'SANDALWOOD', 'CAFFEINE', 'CAFFEINE FREE', 'CAFFEINATED', 'MELON']

    attb_subs.update({'PNUT BUTTER':'PEANUT BUTTER', 'BBQ':'BARBECUE', 'PBUTTER':'PEANUT BUTTER', 'PMPKIN':'PUMPKIN', 'CAFFEINE':'COFFEE', 'CAFFEINE FREE':'', 'CAFFEINATED':'COFFEE', 'STRBRY':'STRAWBERRY', 'GOLD':''})

    # Generate Results
    print(attrib, select_dpt)
    print(f"Total input items: [ {len(items.index)} x {len(items.columns)} ]")
    items['Normalized_'+attrib] = items.DESCRIPTION.replace(replacements, regex = True).apply(lambda x: attribute_fetch(x, color_list_org=attb_list, subs=attb_subs))
    outputs.append(items)

output_df = pd.concat(outputs)
# Save results 
today = dt.datetime.today().strftime("%m%d%Y")
#input_file = list(upload_w.value.values())[0]['metadata']['name']
#input_file = input_file.replace(' ','_').replace('&','_')
output_path = '/dbfs/FileStore/tables/NORMALIZED_ATTRIBUTE_OUTPUTS/'
outfile = 'NormalIzed_Attributes_' + today +'_.csv'

items = output_df.copy()
print('Generating the output and saving the results...')
for i in range(len(items['Normalized_' + attrib].tolist())):
    if items['Normalized_' + attrib].fillna('').iloc[i].find('UNSCENTED') > -1:
        items.at[i, 'Normalized_' + attrib] = 'UNSCENTED'
    if items['Normalized_' + attrib].fillna('').iloc[i].find('FRAGRANCE FREE') > -1:
        items.at[i, 'Normalized_' + attrib] = 'FRAGRANCE FREE'

    if items['DESCRIPTION'].iloc[i].upper().find('GOLD BOND') > -1:
        items.at[i, 'Normalized_' + attrib] = items['Normalized_' + attrib].iloc[i].replace(';GOLD', '')
        items.at[i, 'Normalized_' + attrib] = items['Normalized_' + attrib].iloc[i].replace('GOLD;', '')

    if items['Normalized_' + attrib].fillna('').iloc[i].find(';SCENTED') > -1:
        items.at[i, 'Normalized_' + attrib] = items['Normalized_' + attrib].iloc[i].replace(';SCENTED', '')
    if items['Normalized_' + attrib].fillna('').iloc[i].find('SCENTED;') > -1:
        items.at[i, 'Normalized_' + attrib] = items['Normalized_' + attrib].iloc[i].replace('SCENTED;', '')
    if items['Normalized_' + attrib].fillna('').iloc[i].find('ORIGINAL;') > -1:
        items.at[i, 'Normalized_' + attrib] = items['Normalized_' + attrib].iloc[i].replace('ORIGINAL;', '')
    if items['Normalized_' + attrib].fillna('').iloc[i].find(';ORIGINAL') > -1:
        items.at[i, 'Normalized_' + attrib] = items['Normalized_' + attrib].iloc[i].replace(';ORIGINAL', '')
    if items['Normalized_' + attrib].fillna('').iloc[i].find('CAFFEINE FREE') > -1:
        items.at[i, 'Normalized_' + attrib] = items['Normalized_' + attrib].iloc[i].replace('COFFEE', 'OTHER')
    if items['Normalized_' + attrib].fillna('').iloc[i]=='':
        items.at[i, 'Normalized_' + attrib] = "OTHER"
for attrib in ['Color','Flavor','Scent']:
    items = items.rename(columns={'Normalized_'+attrib : 'Kroger Derived Normalized '+attrib})
items['GTIN'] = items.GTIN.astype(int).astype(str).apply(lambda x: (14- len(x))*'0'+x)
items['Kroger Derived Normalized Diet Type'] = np.nan

cols2keep = ['GTIN','Kroger Derived Normalized Flavor', 'Kroger Derived Normalized Scent','Kroger Derived Normalized Color','Kroger Derived Normalized Diet Type']
items[cols2keep].to_csv(output_path+ outfile, index = None , sep = '|')
print("Done... DBFS Path:  /dbfs/FileStore/tables/NORMALIZED_ATTRIBUTE_OUTPUTS/", outfile)

#########################################################################################

from ipywidgets import HTML
from IPython.display import display
import base64
result = ("http://adb-4812933386228410.10.azuredatabricks.net/files/tables/NORMALIZED_ATTRIBUTE_OUTPUTS/" + outfile)

filename = outfile
b64 = base64.b64encode(result.encode())
payload = b64.decode()

html_buttons = '''<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
</head>
<body>
<a download="{filename}" href="'''+ result +'''" download>
<button class="p-Widget jupyter-widgets jupyter-button widget-button mod-info"><b>Download File</b></button>
</a>
</body>
</html>
'''

html_button = html_buttons.format(payload=payload,filename=filename)
display(HTML(html_button))

print("\nIf the above link doesn't work, try pasting this URL into your browser.\n" + result)

Loading the latest input file: /dbfs/FileStore/tables/NORMALIZED_ATTRIBUTE_INPUTS/SampleInputWeekly_122424-1.csv
Completed loading up configs.
Color Kitchen & Dining
Total input items: [ 476 x 5 ]
Completed loading up configs.
Color Home_Decor_Rugs
Total input items: [ 28 x 5 ]
Completed loading up configs.
Color Home_Decor_NoRugs
Total input items: [ 596 x 5 ]
Completed loading up configs.
Color Floral
Total input items: [ 193 x 5 ]
Completed loading up configs.
Flavor Pantry
Total input items: [ 4807 x 5 ]
Completed loading up configs.
Flavor Beverages
Total input items: [ 1483 x 5 ]
Completed loading up configs.
Flavor Ice Cream
Total input items: [ 248 x 5 ]
Completed loading up configs.
Flavor Candy
Total input items: [ 1403 x 5 ]
Completed loading up configs.
Scent Cleaning and Household
Total input items: [ 934 x 5 ]
Completed loading up configs.
Scent Beauty
Total input items: [ 1641 x 5 ]
Completed loading up configs.
Personal Care
Scent Personal Care
Total input items: [ 1766

HTML(value='<html>\n<head>\n<meta name="viewport" content="width=device-width, initial-scale=1">\n</head>\n<bo…


If the above link doesn't work, try pasting this URL into your browser.
http://adb-4812933386228410.10.azuredatabricks.net/files/tables/NORMALIZED_ATTRIBUTE_OUTPUTS/NormalIzed_Attributes_01252024_.csv
